<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [ ]:
import numpy as np

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [ ]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [ ]:
def get_vocab(corpus):    
    #separamos y en una única lista de términos
    terms = [doc.split() for doc in corpus]
    terms = np.concatenate(terms)
   
    #vector de términos no repetidos
    vocab = np.unique(terms)
    return vocab

In [ ]:
vocab = get_vocab(corpus)
print(vocab)

['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [ ]:
def oneHot(corpus,vocab):    
    one_hot = np.zeros((corpus.size, vocab.size))

    # Recorrer los documentos y marcar las posiciones correspondientes con un valor de 1
    for i, doc in enumerate(corpus):
        terms = doc.split()
        for term in terms:
            j = np.where(vocab == term)[0][0]
            one_hot[i, j] = 1

    return one_hot

In [ ]:
one_hot= oneHot(corpus,vocab)
print(one_hot)

[[0. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 0.]]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [ ]:
def frequency_vectors(corpus, vocab):
    freq_matrix = np.zeros((len(corpus), len(vocab)))

    for i, doc in enumerate(corpus):
        terms = doc.split()
        term_counts = np.bincount([np.where(vocab == term)[0][0] for term in terms if term in vocab])
        freq_matrix[i, :len(term_counts)] = term_counts

    return freq_matrix

freq_matrix = frequency_vectors(corpus, vocab)
print(freq_matrix)


[[0. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0. 1. 2. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 0.]]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [ ]:
def tfidf_representation(corpus):
    # Obtener el vocabulario
    vocab = get_vocab(corpus)

    # Obtener la matriz de frecuencia
    freq_matrix = frequency_vectors(corpus, vocab)

    # Calcular IDF (Inverse Document Frequency)
    doc_freq = np.sum(freq_matrix > 0, axis=0)
    idf = np.log(len(corpus) / (1 + doc_freq))

    # Calcular TF-IDF
    tfidf_matrix = freq_matrix * idf

    return tfidf_matrix

tfidf_matrix = tfidf_representation(corpus)
print(tfidf_matrix)


[[0.         0.         0.         0.         0.         0.
  0.         0.         0.40546511]
 [0.40546511 0.         0.40546511 0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.40546511 0.
  0.         0.40546511 0.        ]]


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [ ]:
def document_similarity(corpus, doc_index):
    # Obtener el vocabulario y la matriz TF-IDF del corpus completo
    vocab = get_vocab(corpus)
    tfidf_matrix = tfidf_representation(corpus)

    # Calcular la similitud coseno entre el documento de interés y los demás documentos
    doc_tfidf = tfidf_matrix[doc_index]
    similarities = []
    for i in range(len(corpus)):
        if i != doc_index:
            similarity = cosine_similarity(doc_tfidf, tfidf_matrix[i])
            similarities.append((i, similarity))

    # Ordenar los documentos por similitud coseno en orden descendente
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Obtener los índices de los documentos ordenados
    ordered_indices = [similarity[0] for similarity in similarities]

    # Devolver los documentos ordenados por similitud coseno
    ordered_documents = [corpus[index] for index in ordered_indices]

    return ordered_documents


In [ ]:
    for idx in range(len(corpus)):
    ordered_documents = document_similarity(corpus, idx)
    print(ordered_documents)

['martes el dia de hoy es martes', 'martes muchas gracias']
['que dia es hoy', 'martes muchas gracias']
['que dia es hoy', 'martes el dia de hoy es martes']
